In [1]:
# Import packages
import random
import re
import numpy as np

import gensim
import gensim.downloader

#!pip uninstall nltk
#!pip install -U nltk
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

# Load the Google News Word2Vec model
w2vModel = gensim.downloader.load("word2vec-google-news-300")

from scipy.spatial import distance

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mrpie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# The target length for a line (in words)
TARGET_SENTENCE_LENGTH = 7
# Which words will count as unstressed syllables
UNSTRESSED_WORDS = ['A', 'AN', 'THE', 'TO', 'OF']

# Weights for the linear combinations for the 4 ways words are scored
RANDOM_WEIGHT = 0.1
RHYME_WEIGHT = 0.3
BIGRAM_WEIGHT = 0.2
CLOSENESS_WEIGHT = 0.4

In [3]:
# A syllable, used to calculate rhyme and meter
class Syllable:
  def __init__(self, vowel, consonants, stress):
    self.vowel = vowel
    self.consonants = "".join(consonants)
    self.stress = stress
  
  # Print the syllable as a string capitalized to show the stress.
  def __repr__(self):
    vowel = self.vowel
    consonants = self.consonants
    if self.stress == 0:
      vowel = self.vowel.lower()
    if self.stress != 1:
      consonants = self.consonants.lower()
    return vowel + consonants
  
  # Score this syllable by how much it rhymes with another one using stress and vowel
  def rhyme_score(self, other):
    score = 0
    if self.stress == other.stress:
      score += 1
      if self.vowel == other.vowel:
        score += 2
        if self.consonants == other.consonants:
          score += 3
    elif self.stress > 0 and other.stress > 0 and self.vowel == other.vowel:
      score += 1
      if self.consonants == other.consonants:
        score += 3
    return score

In [4]:
# A word, used for rhymes. Comrpised of syllables.
# Words can have multiple pronunciations in the CMU pronouncing dictionary
class Word:
  is_vowel = re.compile('.*[0-2]')
  
  # Create this word
  def __init__(self, word=None, syllables=None, stressed=True):
    self.word = word
    self.pronunciations = []
    self.stressed = stressed
    if word is not None and syllables is not None:
      self.add_pronunciation(syllables)
      self.length = len(self.pronunciations[0])
    else:
      self.length = 0
  
  # String representation for debugging
  def __repr__(self):
    list = []
    for syllable in self.pronunciations[0]:
      list.append(str(syllable))
    for pron in self.pronunciations[1:]:
      list.append("|")
      for syllable in pron:
        list.append(str(syllable))
    return self.word + ": " + "".join(list)
  
  # Parse the given string into syllables and add to pronunciations
  def add_pronunciation(self, syllables):
    pronunciation = []
    first_idx = 0
    while not Word.is_vowel.match(syllables[first_idx]):
      first_idx += 1
    
    vowel = syllables[first_idx][0:-1]
    stress = int(syllables[first_idx][-1])
    consonants = []
    first_idx += 1
    for i in range(first_idx, len(syllables)):
      if Word.is_vowel.match(syllables[i]):
        pronunciation.append(Syllable(vowel, consonants, stress))
        if not self.stressed:
          pronunciation[-1].stress = 0
        vowel = syllables[i][0:-1]
        stress = int(syllables[i][-1])
        consonants = []
      else:
        consonants.append(syllables[i])
    pronunciation.append(Syllable(vowel, consonants, stress))
    self.pronunciations.append(pronunciation)

    # Deprecated rhyme function
#   def rhyme_score(self, others):
#     syllables = self.length
#     word_offset = len(others) - 1
#     syl_offset = 0
#     max_score = 0
#     while syllables > 0:
#       if word_offset < 0:
#         return 0, -1
#       word = others[word_offset]
#       word_offset -= 1
#       word_score = 0
#       best_pron = 0
#       for i in range(len(self.pronunciations)):
#         next_pron = self.pronunciations[i][:syllables]
#         pron = reversed(next_pron)
#         for j in range(len(word.pronunciations)):
#           score = 0
#           o_pron = reversed(word.pronunciations[j])
#           for syl1, syl2 in zip(pron, o_pron):
#             score += syl1.rhyme_score(syl2)
#           if score >= word_score:
#             word_score = score
#             best_pron = j
#       max_score += word_score
#       syllables -= word.length
#     return max_score, best_pron
  
  # Return an identical copy of this word for calculating rhymes
  def copy(self):
    my_copy = Word()
    for pron in self.pronunciations:
      new_pron = []
      for syl in pron:
        new_pron.append(syl)
      my_copy.pronunciations.append(new_pron)
    my_copy.length = self.length
    my_copy.word = self.word
    return my_copy
  
  # Chop off the given number of syllables from the end of this word.
  # Used for calculating rhymes from many-to-many words
  def chop(self, syllables):
    for i in range(0, len(self.pronunciations)):
      self.pronunciations[i] = self.pronunciations[i][0:-syllables]
    self.length -= syllables

In [5]:
# Return a rhyme score between the two lists of words.
# Calculates rhyme based on syllables while considering different pronunciations of each word.
# Returns the score for the most-rhyming pronunciation for each word
# score between 'tree house' and 'bee blouse' is the same as 'treehouse' and 'bee blouse' (disregarding differing meters)
def score_rhyme(words1, words2):
  idx1 = len(words1) - 1
  idx2 = len(words2) - 1
  total_score = 0
  word1 = Word()
  word2 = Word()
  while (word1.length > 0 or idx1 >= 0) and (word2.length > 0 or idx2 >= 0):
    if word1.length == 0:
      word1 = words1[idx1].copy()
    syl1 = []
    for pron in word1.pronunciations:
      syl1.append(list(reversed(pron)))
    idx1 -= 1
    if word2.length == 0:
      syl2 = []
    word2 = words2[idx2].copy()
    for pron in word2.pronunciations:
      syl2.append(list(reversed(pron)))
    idx2 -= 1
    max_score = 0
    for pron1 in syl1:
      for pron2 in syl2:
        score = 0
        for s1, s2 in zip(pron1, pron2):
          score += s1.rhyme_score(s2)
        if score > max_score:
          max_score = score
    total_score += max_score
    consumed = min(word1.length, word2.length)
    word1.chop(consumed)
    word2.chop(consumed)
  return total_score

In [6]:
# Read the CMU pronouncing dictionary (http://www.speech.cs.cmu.edu/cgi-bin/cmudict)
def parse_syllable_file():
  f = open('data/cmudict-0.7b', 'r') #, encoding = "ISO-8859-1")
  duplicate = re.compile(r'(.*)\([0-9]+\)')
  words = {}
  for line in f:
    if not line.startswith(';;;'):
      line = line.strip().split(' ')
      if duplicate.match(line[0]):
        word = duplicate.split(line[0])[1]
        words[word].add_pronunciation(line[1:])
      else:
        word = Word(line[0], line[1:], line[0] in UNSTRESSED_WORDS)
        words[line[0]] = word
          
  return words

In [7]:
# A node in the generative context-free grammar
# Has productions which can be other nodes or terminals (words)
class ParseNode:
  # Every ParseNode, stored by its name
  grammar = {}
  # Every terminal, stored in lists under their parent's name
  terminals = {}
  
  # Read the given grammar file and construct a tree of nodes
  def parse_grammar(filename):
    file = open(filename, 'r')
    node = ParseNode(file.readline().strip())
    ParseNode.root = node
    ParseNode.grammar[node.name] = node
    for line in file:
      if line.startswith(' '):
        children = line.strip().split(' ')
        for i in range(0, len(children)):
          if children[i] in ParseNode.grammar:
            children[i] = ParseNode.grammar[children[i]]
          else:
            children[i] = ParseNode(children[i])
            ParseNode.grammar[children[i].name] = children[i]
        node.productions.append(children)
      else:
        name = line.strip()
        if name in ParseNode.grammar:
          node = ParseNode.grammar[name]
        else:
          node = ParseNode(name)
          ParseNode.grammar[name] = node
    file.close()
    
    # Approximate the length of a production over 3 iterations
    for i in range(3):
      for key in ParseNode.grammar:
        ParseNode.grammar[key].calculate_avg_length()
    
    # Load all the terminals and associate them with their ParseNode
    terminals = open('data/terminals.txt', 'r')
    for line in terminals:
      tokens = line.strip().split(' ')
      if len(tokens) == 2 and tokens[1] in ParseNode.grammar:
        if tokens[1] in ParseNode.terminals:
          ParseNode.terminals[tokens[1]].append(tokens[0])
        else:
          ParseNode.terminals[tokens[1]] = [tokens[0]]
    terminals.close()
  
  # Generate a legal production using the CGF
  def generate_text(oracle, rhyme_with=None):
    # The text is a list, which is a mixture of ParseNodes and strings
    text = [ParseNode.root]
    generating = True
    # Prepare the rhyming words by copying the words in rhyme_with
    rhyme_syllables = []
    for word in rhyme_with:
      if word.upper() in rhymes:
        rhyme_syllables.append(rhymes[word.upper()].copy())
      else:
        break
    rhyme_with = rhyme_syllables
    # Until there are no more non-terminals
    # Generate a legal production from the rightmost non-terminal (helps with rhyme)
    while generating:
      generating = False
      # Iterate over the list backwards
      for i in range(len(text) - 1, -1, -1):
        # If a non-terminal, expand it.
        if isinstance(text[i], ParseNode):
          generating = True
          next_word = None
          if i+1 < len(text) and not isinstance(text[i+1], ParseNode):
            next_word = text[i+1]
          production = text[i].make_production(oracle, text, i, rhyme_with)
          text = text[0:i] + production + text[i+1:]
          if not isinstance(text[i], ParseNode):
            if text[i].upper() in rhymes:
              word = rhymes[text[i].upper()]
              num_sylls = word.length
              while num_sylls > 0 and len(rhyme_with) > 0:
                rhyme_word = rhyme_with.pop()
                if rhyme_word.length > num_sylls:
                  rhyme_word.chop(num_sylls)
                  rhyme_with.append(rhyme_word)
                  num_sylls = 0
                else:
                  num_sylls -= rhyme_word.length
            else:
              rhyme_with = []
          break
    return text
  
  # Construct a ParseNode
  def __init__(self, name):
    self.name = name
    self.productions = []
    self.avg_length = 0
  
  # String representation for debugging ('NT' stands for nonterminal)
  def __repr__(self):
    return 'NT(' + self.name + ')'
  
  # Expand into children in the parse tree
  def make_production(self, oracle, text, index, rhyme_with):
    # Keep track of the updating distance to the original input vector
    global SONG_VECTOR, TOTAL_DISTANCE
    # If this nonterminal only has terminals as leaves
    if len(self.productions) == 0:
      max_score = -1
      production = None
      best_vector = None
      best_distance = 0
      # Check each terminal to see if it moves the vector closer to the input text
      # As well as score it using the oracle to check for rhyme/presence in the lyric corpus
      for terminal in ParseNode.terminals[self.name]:
        score = oracle(terminal, text, index, rhyme_with)
        closeness, new_vector, distance_with_word = score_semantics(terminal)
        score += closeness * CLOSENESS_WEIGHT + random.random() * RANDOM_WEIGHT
        if score > max_score:
          max_score = score
          production = terminal
          best_vector = new_vector
          best_distance = distance_with_word
      SONG_VECTOR = best_vector
      TOTAL_DISTANCE = best_distance
      return [production]
    else:
      # Expand into non-terminal children. Check each child using the oracle to ensure the line isn't too big or small.
      production = self.productions[0]
      max_score = oracle(production, text, index, rhyme_with)
      for prod in self.productions[1:]:
        score = oracle(prod, text, index, rhyme_with)
        if score > max_score:
          production = prod
          max_score = score
      return production
  
  # Used when iteratively calculating average production length in words
  # Uses the average length of children. This changes each iteration due to children being nonterminals
  def calculate_avg_length(self):
    if len(self.productions) == 0:
      self.avg_length = 1
    else:
      for prod in self.productions:
        self.avg_length += len(prod)
      self.avg_length /= len(self.productions)

In [8]:
# How much closer the next word puts the average song vector to the average input vector
def score_semantics(next_word):
  if next_word in w2vModel:
    new_vector = np.add(SONG_VECTOR, w2vModel[next_word])

    #euclidean distance
    distance_with_word = distance.cosine(REFERENCE_VECTOR, new_vector)

    return (TOTAL_DISTANCE - distance_with_word) / TOTAL_DISTANCE, new_vector, distance_with_word
  return 0, SONG_VECTOR, TOTAL_DISTANCE

In [9]:
# Score the given production
def oracle(production, text, index, rhyme_with):
  # If the production is made of ParseNodes
  if isinstance(production[0], ParseNode):
    # Calculate the predicted length and score based on distance to the target length
    # Along with some random noise for variability
    predicted_length = 0
    for node in text:
      if isinstance(node, ParseNode):
        predicted_length += node.avg_length
      else:
        predicted_length += 1
    for node in production:
      predicted_length += node.avg_length
    score = TARGET_SENTENCE_LENGTH - abs(predicted_length - TARGET_SENTENCE_LENGTH)
    return score + 2 * random.random()
  else:
    # Score the word based on rhyme and presence in the lyric corpus
    word = production
    pron_idx = -1
    rhyme_score = 0
    if word.upper() in rhymes:
      if len(rhyme_with) > 0 and word.upper() == rhyme_with[-1].word:
        rhyme_score = 0
      else:
        rhyme_score = score_rhyme([rhymes[word.upper()]], rhyme_with)
        rhyme_score /= (rhymes[word.upper()].length * 6)
    if index + 1 >= len(text):
      return rhyme_score * RHYME_WEIGHT
    else:
      next_word = text[index + 1]
      bigram = word + " " + next_word
      score = 0
      if bigram in bigrams:
        score = 1
      else:
        score = 0
      return score * BIGRAM_WEIGHT + rhyme_score * RHYME_WEIGHT

In [10]:
# Load the bigrams from the lyric corpus
bigrams = {}
bigram_file = open('data/bigrams.txt', 'r')
# Load the CMU pronouncing dictionary
rhymes = parse_syllable_file()
for line in bigram_file:
  tokens = line.strip().split('\t')
  if len(tokens) == 2:
    bigrams[tokens[0]] = int(tokens[1])
# Load the grammar from grammar.txt
ParseNode.parse_grammar('data/grammar.txt')

In [11]:
# Generate a song using AABB rhyme scheme.
# Each verse is 2 lines. Each line is a production from the CFG.
def generate_song(num_verses):
  for i in range(num_verses):
    l1 = ParseNode.generate_text(oracle, rhyme_with='')
    l2 = ParseNode.generate_text(oracle, rhyme_with='')
    print(' '.join(l1))
    print(' '.join(ParseNode.generate_text(oracle, rhyme_with=l1)))
    print(' '.join(l2))
    print(' '.join(ParseNode.generate_text(oracle, rhyme_with=l2)))
    print()

In [12]:
# "reference" refers to the input text
# Generate a song based on the given input text and number of verses
reference = []

line = input('Write some text, empty line to generate song\n')
while line != '':
  #line = input()
  reference.append(line)
  line = input()

reference = '\n'.join(reference).lower()
reference_tokens = word_tokenize(reference)

REFERENCE_VECTOR = np.zeros((len(reference_tokens), 300))

for i in range(len(reference_tokens)):
  if reference_tokens[i] in w2vModel:
    REFERENCE_VECTOR[i] = w2vModel[reference_tokens[i]]

REFERENCE_VECTOR = np.sum(REFERENCE_VECTOR, axis=0)
SONG_VECTOR = np.zeros((300))
TOTAL_DISTANCE = 1
num_verses = int(input('How many verses? '))

print('\n')
print('Generated lyrics:')
generate_song(num_verses)

Write some text, empty line to generate song
 I wake up and make bacon
 
How many verses?  5




Generated lyrics:
anyway dogg shawty make it
please lamb spark carry da
waking ups pie needs grey
guns got one day

might texas woke itself
may christmas smoke yourself
these butter others feed july
yours best is i

st realized oooh
that make thangs followed saturday
leading blunts grown make
might deepest thugs know they

all connection sent jordan
will an cent keep them
lamb stops both jeans
an pants part hoes meat

beyond the bed blunts
ought de lets us
nicki easy listen cha
nah freaky story open tops



In [13]:
# def data_add(word1, word2):
#   word1 = rhymes[word1.upper()]
#   word2 = rhymes[word2.upper()]
#   data['Word 1'].append(str(word1))
#   data['Word 2'].append(str(word2))
#   data['Score'].append(score_rhyme([word1], [word2]))

# data = {'Word 1': [], 'Word 2': [], 'Score': []}
# data_add('rent', 'box')
# data_add('rent', 'bed')
# data_add('rent', 'bent')
# data_add('rent', 'frequent')
# data_add('rent', 'descent')
# import pandas
# pandas.DataFrame(data)